In [3]:
import scipy.io as sio
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

def tSVD(tensor, rank):
    """ Tensor Singular Value Decomposition """
    unfold_tensor = tensor.reshape(tensor.shape[0], -1)
    U, S, VT = np.linalg.svd(unfold_tensor, full_matrices=False)
    U = U[:, :rank]
    S = np.diag(S)[:rank, :rank]
    VT = VT[:rank, :]
    return U, S, VT

def tProduct(U, S, VT):
    """ Tensor Product """
    result = np.tensordot(U, S, axes=(1, 0))
    result = np.tensordot(result, VT, axes=(1, 0))
    return result

# Load the provided dataset
data = sio.loadmat("C:/Users/Mrlaptop/Downloads/Brain_Data_Mat_Format/Brain_Data_Mat_Format/008-01reflectance.mat")

# Inspect the keys to understand the structure of the data
print(data.keys())

# Assuming the dataset has similar structure
# img_gt = data['some_key_for_ground_truth']
# img = data['some_key_for_image']

# Update this with actual keys after inspecting the structure
img = data['reflectance']

# Example placeholder for ground truth
W, H, B = img.shape
img_gt = np.random.randint(1, 5, (W, H))  # Assuming there are 4 classes

# Parameters (Can be changed as required)
u = 5
w = 2 * u + 1
w2 = w * w
L = 49

# TensorSSA
indian_pines = np.pad(img, ((u, u), (u, u), (0, 0)), mode='symmetric')
Id = np.zeros((L, W * H), dtype=int)
Fea_cube = np.zeros((L, W * H, B))

# Adaptive embedding
k = 0
for i in range(W):
    for j in range(H):
        i1 = i + u
        j1 = j + u
        k += 1
        testcube = indian_pines[i1-u:i1+u+1, j1-u:j1+u+1, :]
        m = testcube.reshape(w2, B)

        # NED
        center = m[(w2+1)//2, :]
        NED = np.sqrt(np.sum(((m / np.linalg.norm(m, axis=1, keepdims=True)) - (center / np.linalg.norm(center)))**2, axis=1))
        ind = np.argsort(NED)
        index = ind[:L]
        Id[:, k-1] = index
        Fea_cube[:, k-1, :] = m[index, :]

# T-SVD decomposition
rank = min(L, B)  # Ensure the rank is appropriate for the tensor dimensions
U, S, VT = tSVD(Fea_cube, rank)

# Ensure shapes are compatible for tProduct
print(f"Shapes before tProduct: U={U.shape}, S={S.shape}, VT={VT.shape}")
assert U.shape[1] == S.shape[0]
assert S.shape[1] == VT.shape[0]

C = tProduct(U, S, VT)
print(f"Shape after first tProduct (C): {C.shape}")

# Perform the final tProduct
Feacube_proc = C.reshape(L, W * H, B)
print(f"Shape after second tProduct (Feacube_proc): {Feacube_proc.shape}")

# SVM and RF based classification
Labels = img_gt.reshape(W*H)
Vectors = Feacube_proc.transpose(1, 0, 2).reshape(W*H, L*B)  # Use processed feature cube for SVM and RF

class_num = np.max(img_gt) - np.min(img_gt)
trainVectors, trainLabels, testVectors, testLabels = [], [], [], []

rng = np.random.default_rng()
Sam = 0.02  # Training sample ratio: 0.02 for IP, 0.01 for PU and MG

for k in range(1, class_num + 1):
    index = np.where(Labels == k)[0]
    perclass_num = len(index)
    Vectors_perclass = Vectors[index, :]
    c = rng.permutation(perclass_num)
    select_train = Vectors_perclass[c[:int(np.ceil(perclass_num * Sam))], :]
    train_index_k = index[c[:int(np.ceil(perclass_num * Sam))]]
    trainVectors.append(select_train)
    trainLabels.extend([k] * int(np.ceil(perclass_num * Sam)))

    select_test = Vectors_perclass[c[int(np.ceil(perclass_num * Sam)):], :]
    test_index_k = index[c[int(np.ceil(perclass_num * Sam)):]]
    testVectors.append(select_test)
    testLabels.extend([k] * (perclass_num - int(np.ceil(perclass_num * Sam))))

trainVectors = np.vstack(trainVectors)
testVectors = np.vstack(testVectors)

# Feature scaling
scaler = StandardScaler().fit(trainVectors)
trainVectors_scaled = scaler.transform(trainVectors)
testVectors_scaled = scaler.transform(testVectors)
Vectors_scaled = scaler.transform(Vectors)

# SVM classification
svm_classifier = SVC(kernel='linear', C=1)
svm_classifier.fit(trainVectors_scaled, trainLabels)
predicted_labels_svm = svm_classifier.predict(testVectors_scaled)

# RF classification
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(trainVectors_scaled, trainLabels)
predicted_labels_rf = rf_classifier.predict(testVectors_scaled)

# Evaluate the performance
conf_matrix_svm = confusion_matrix(testLabels, predicted_labels_svm)
accuracy_svm = accuracy_score(testLabels, predicted_labels_svm)
kappa_svm = cohen_kappa_score(testLabels, predicted_labels_svm)

conf_matrix_rf = confusion_matrix(testLabels, predicted_labels_rf)
accuracy_rf = accuracy_score(testLabels, predicted_labels_rf)
kappa_rf = cohen_kappa_score(testLabels, predicted_labels_rf)

print(f"SVM Confusion Matrix:\n{conf_matrix_svm}")
print(f"SVM Accuracy: {accuracy_svm}")
print(f"SVM Kappa: {kappa_svm}")

print(f"RF Confusion Matrix:\n{conf_matrix_rf}")
print(f"RF Accuracy: {accuracy_rf}")
print(f"RF Kappa: {kappa_rf}")


NotImplementedError: Please use HDF reader for matlab v7.3 files

In [5]:
import h5py
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import matplotlib.pyplot as plt

def tSVD(tensor, rank):
    """ Tensor Singular Value Decomposition """
    unfold_tensor = tensor.reshape(tensor.shape[0], -1)
    U, S, VT = np.linalg.svd(unfold_tensor, full_matrices=False)
    U = U[:, :rank]
    S = np.diag(S)[:rank, :rank]
    VT = VT[:rank, :]
    return U, S, VT

def tProduct(U, S, VT):
    """ Tensor Product """
    result = np.tensordot(U, S, axes=(1, 0))
    result = np.tensordot(result, VT, axes=(1, 0))
    return result

# Load the provided dataset
file_path = "C:/Users/Mrlaptop/Downloads/Brain_Data_Mat_Format/Brain_Data_Mat_Format/008-01reflectance.mat"

with h5py.File(file_path, 'r') as f:
    # Assuming the dataset key is 'reflectance'
    # Update this with the actual key after inspecting the structure
    img = f['reflectance'][:]

# Example placeholder for ground truth
W, H, B = img.shape
img_gt = np.random.randint(1, 5, (W, H))  # Assuming there are 4 classes

# Parameters (Can be changed as required)
u = 5
w = 2 * u + 1
w2 = w * w
L = 49

# TensorSSA
indian_pines = np.pad(img, ((u, u), (u, u), (0, 0)), mode='symmetric')
Id = np.zeros((L, W * H), dtype=int)
Fea_cube = np.zeros((L, W * H, B))

# Adaptive embedding
k = 0
for i in range(W):
    for j in range(H):
        i1 = i + u
        j1 = j + u
        k += 1
        testcube = indian_pines[i1-u:i1+u+1, j1-u:j1+u+1, :]
        m = testcube.reshape(w2, B)

        # NED
        center = m[(w2+1)//2, :]
        NED = np.sqrt(np.sum(((m / np.linalg.norm(m, axis=1, keepdims=True)) - (center / np.linalg.norm(center)))**2, axis=1))
        ind = np.argsort(NED)
        index = ind[:L]
        Id[:, k-1] = index
        Fea_cube[:, k-1, :] = m[index, :]

# T-SVD decomposition
rank = min(L, B)  # Ensure the rank is appropriate for the tensor dimensions
U, S, VT = tSVD(Fea_cube, rank)

# Ensure shapes are compatible for tProduct
print(f"Shapes before tProduct: U={U.shape}, S={S.shape}, VT={VT.shape}")
assert U.shape[1] == S.shape[0]
assert S.shape[1] == VT.shape[0]

C = tProduct(U, S, VT)
print(f"Shape after first tProduct (C): {C.shape}")

# Perform the final tProduct
Feacube_proc = C.reshape(L, W * H, B)
print(f"Shape after second tProduct (Feacube_proc): {Feacube_proc.shape}")

# SVM and RF based classification
Labels = img_gt.reshape(W*H)
Vectors = Feacube_proc.transpose(1, 0, 2).reshape(W*H, L*B)  # Use processed feature cube for SVM and RF

class_num = np.max(img_gt) - np.min(img_gt)
trainVectors, trainLabels, testVectors, testLabels = [], [], [], []

rng = np.random.default_rng()
Sam = 0.02  # Training sample ratio: 0.02 for IP, 0.01 for PU and MG

for k in range(1, class_num + 1):
    index = np.where(Labels == k)[0]
    perclass_num = len(index)
    Vectors_perclass = Vectors[index, :]
    c = rng.permutation(perclass_num)
    select_train = Vectors_perclass[c[:int(np.ceil(perclass_num * Sam))], :]
    train_index_k = index[c[:int(np.ceil(perclass_num * Sam))]]
    trainVectors.append(select_train)
    trainLabels.extend([k] * int(np.ceil(perclass_num * Sam)))

    select_test = Vectors_perclass[c[int(np.ceil(perclass_num * Sam)):], :]
    test_index_k = index[c[int(np.ceil(perclass_num * Sam)):]]
    testVectors.append(select_test)
    testLabels.extend([k] * (perclass_num - int(np.ceil(perclass_num * Sam))))

trainVectors = np.vstack(trainVectors)
testVectors = np.vstack(testVectors)

# Feature scaling
scaler = StandardScaler().fit(trainVectors)
trainVectors_scaled = scaler.transform(trainVectors)
testVectors_scaled = scaler.transform(testVectors)
Vectors_scaled = scaler.transform(Vectors)

# SVM classification
svm_classifier = SVC(kernel='linear', C=1)
svm_classifier.fit(trainVectors_scaled, trainLabels)
predicted_labels_svm = svm_classifier.predict(testVectors_scaled)

# RF classification
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(trainVectors_scaled, trainLabels)
predicted_labels_rf = rf_classifier.predict(testVectors_scaled)

# Evaluate the performance
conf_matrix_svm = confusion_matrix(testLabels, predicted_labels_svm)
accuracy_svm = accuracy_score(testLabels, predicted_labels_svm)
kappa_svm = cohen_kappa_score(testLabels, predicted_labels_svm)

conf_matrix_rf = confusion_matrix(testLabels, predicted_labels_rf)
accuracy_rf = accuracy_score(testLabels, predicted_labels_rf)
kappa_rf = cohen_kappa_score(testLabels, predicted_labels_rf)

print(f"SVM Confusion Matrix:\n{conf_matrix_svm}")
print(f"SVM Accuracy: {accuracy_svm}")
print(f"SVM Kappa: {kappa_svm}")

print(f"RF Confusion Matrix:\n{conf_matrix_rf}")
print(f"RF Accuracy: {accuracy_rf}")
print(f"RF Kappa: {kappa_rf}")


OSError: Unable to open file (truncated file: eof = 46974730, sblock->base_addr = 512, stored_eof = 473815497)

In [8]:
import h5py
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import matplotlib.pyplot as plt

def tSVD(tensor, rank):
    """ Tensor Singular Value Decomposition """
    unfold_tensor = tensor.reshape(tensor.shape[0], -1)
    U, S, VT = np.linalg.svd(unfold_tensor, full_matrices=False)
    U = U[:, :rank]
    S = np.diag(S)[:rank, :rank]
    VT = VT[:rank, :]
    return U, S, VT

def tProduct(U, S, VT):
    """ Tensor Product """
    result = np.tensordot(U, S, axes=(1, 0))
    result = np.tensordot(result, VT, axes=(1, 0))
    return result

# Load the provided dataset
file_path =  "C:/Users/Mrlaptop/Downloads/Brain_Data_Mat_Format/Brain_Data_Mat_Format/008-01reflectance.mat"

try:
    with h5py.File(file_path, 'r') as f:
        # Inspecting the keys to understand the structure of the data
        print(f.keys())
        # Assuming the dataset key is 'reflectance'
        # Update this with the actual key after inspecting the structure
        data = {key: np.array(f[key]) for key in f.keys()}
except Exception as e:
    print(f"Error loading the file: {e}")

# Ensure that the data was loaded
if 'reflectance' in data:
    img = data['reflectance']
else:
    raise KeyError("The key 'reflectance' was not found in the data. Available keys are: ", data.keys())

# Example placeholder for ground truth
W, H, B = img.shape
img_gt = np.random.randint(1, 5, (W, H))  # Assuming there are 4 classes

# Parameters (Can be changed as required)
u = 5
w = 2 * u + 1
w2 = w * w
L = 49

# TensorSSA
indian_pines = np.pad(img, ((u, u), (u, u), (0, 0)), mode='symmetric')
Id = np.zeros((L, W * H), dtype=int)
Fea_cube = np.zeros((L, W * H, B))

# Adaptive embedding
k = 0
for i in range(W):
    for j in range(H):
        i1 = i + u
        j1 = j + u
        k += 1
        testcube = indian_pines[i1-u:i1+u+1, j1-u:j1+u+1, :]
        m = testcube.reshape(w2, B)

        # NED
        center = m[(w2+1)//2, :]
        NED = np.sqrt(np.sum(((m / np.linalg.norm(m, axis=1, keepdims=True)) - (center / np.linalg.norm(center)))**2, axis=1))
        ind = np.argsort(NED)
        index = ind[:L]
        Id[:, k-1] = index
        Fea_cube[:, k-1, :] = m[index, :]

# T-SVD decomposition
rank = min(L, B)  # Ensure the rank is appropriate for the tensor dimensions
U, S, VT = tSVD(Fea_cube, rank)

# Ensure shapes are compatible for tProduct
print(f"Shapes before tProduct: U={U.shape}, S={S.shape}, VT={VT.shape}")
assert U.shape[1] == S.shape[0]
assert S.shape[1] == VT.shape[0]

C = tProduct(U, S, VT)
print(f"Shape after first tProduct (C): {C.shape}")

# Perform the final tProduct
Feacube_proc = C.reshape(L, W * H, B)
print(f"Shape after second tProduct (Feacube_proc): {Feacube_proc.shape}")

# SVM and RF based classification
Labels = img_gt.reshape(W*H)
Vectors = Feacube_proc.transpose(1, 0, 2).reshape(W*H, L*B)  # Use processed feature cube for SVM and RF

class_num = np.max(img_gt) - np.min(img_gt)
trainVectors, trainLabels, testVectors, testLabels = [], [], [], []

rng = np.random.default_rng()
Sam = 0.02  # Training sample ratio: 0.02 for IP, 0.01 for PU and MG

for k in range(1, class_num + 1):
    index = np.where(Labels == k)[0]
    perclass_num = len(index)
    Vectors_perclass = Vectors[index, :]
    c = rng.permutation(perclass_num)
    select_train = Vectors_perclass[c[:int(np.ceil(perclass_num * Sam))], :]
    train_index_k = index[c[:int(np.ceil(perclass_num * Sam))]]
    trainVectors.append(select_train)
    trainLabels.extend([k] * int(np.ceil(perclass_num * Sam)))

    select_test = Vectors_perclass[c[int(np.ceil(perclass_num * Sam)):], :]
    test_index_k = index[c[int(np.ceil(perclass_num * Sam)):]]
    testVectors.append(select_test)
    testLabels.extend([k] * (perclass_num - int(np.ceil(perclass_num * Sam))))

trainVectors = np.vstack(trainVectors)
testVectors = np.vstack(testVectors)

# Feature scaling
scaler = StandardScaler().fit(trainVectors)
trainVectors_scaled = scaler.transform(trainVectors)
testVectors_scaled = scaler.transform(testVectors)
Vectors_scaled = scaler.transform(Vectors)

# SVM classification
svm_classifier = SVC(kernel='linear', C=1)
svm_classifier.fit(trainVectors_scaled, trainLabels)
predicted_labels_svm = svm_classifier.predict(testVectors_scaled)

# RF classification
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(trainVectors_scaled, trainLabels)
predicted_labels_rf = rf_classifier.predict(testVectors_scaled)

# Evaluate the performance
conf_matrix_svm = confusion_matrix(testLabels, predicted_labels_svm)
accuracy_svm = accuracy_score(testLabels, predicted_labels_svm)
kappa_svm = cohen_kappa_score(testLabels, predicted_labels_svm)

conf_matrix_rf = confusion_matrix(testLabels, predicted_labels_rf)
accuracy_rf = accuracy_score(testLabels, predicted_labels_rf)
kappa_rf = cohen_kappa_score(testLabels, predicted_labels_rf)

print(f"SVM Confusion Matrix:\n{conf_matrix_svm}")
print(f"SVM Accuracy: {accuracy_svm}")
print(f"SVM Kappa: {kappa_svm}")

print(f"RF Confusion Matrix:\n{conf_matrix_rf}")
print(f"RF Accuracy: {accuracy_rf}")
print(f"RF Kappa: {kappa_rf}")


Error loading the file: Unable to open file (truncated file: eof = 46974730, sblock->base_addr = 512, stored_eof = 473815497)


NameError: name 'data' is not defined

In [9]:
import h5py
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import matplotlib.pyplot as plt

def tSVD(tensor, rank):
    """ Tensor Singular Value Decomposition """
    unfold_tensor = tensor.reshape(tensor.shape[0], -1)
    U, S, VT = np.linalg.svd(unfold_tensor, full_matrices=False)
    U = U[:, :rank]
    S = np.diag(S)[:rank, :rank]
    VT = VT[:rank, :]
    return U, S, VT

def tProduct(U, S, VT):
    """ Tensor Product """
    result = np.tensordot(U, S, axes=(1, 0))
    result = np.tensordot(result, VT, axes=(1, 0))
    return result

# Load the provided dataset
file_path =  "C:/Users/Mrlaptop/Downloads/Brain_Data_Mat_Format/Brain_Data_Mat_Format/008-01reflectance.mat"


data = {}
try:
    with h5py.File(file_path, 'r') as f:
        # Inspecting the keys to understand the structure of the data
        print(f.keys())
        # Assuming the dataset key is 'reflectance'
        # Update this with the actual key after inspecting the structure
        data = {key: np.array(f[key]) for key in f.keys()}
except Exception as e:
    print(f"Error loading the file: {e}")

# Ensure that the data was loaded
if 'reflectance' in data:
    img = data['reflectance']
else:
    print("Using mock data for img since the file could not be loaded.")
    img = np.random.rand(100, 100, 200)  # Mock data as an example
    img_gt = np.random.randint(1, 5, (100, 100))  # Mock ground truth

# If img_gt is not already defined (from the file), define a mock ground truth
if 'img_gt' not in locals():
    W, H, B = img.shape
    img_gt = np.random.randint(1, 5, (W, H))  # Assuming there are 4 classes

# Parameters (Can be changed as required)
u = 5
w = 2 * u + 1
w2 = w * w
L = 49

# TensorSSA
indian_pines = np.pad(img, ((u, u), (u, u), (0, 0)), mode='symmetric')
Id = np.zeros((L, W * H), dtype=int)
Fea_cube = np.zeros((L, W * H, B))

# Adaptive embedding
k = 0
for i in range(W):
    for j in range(H):
        i1 = i + u
        j1 = j + u
        k += 1
        testcube = indian_pines[i1-u:i1+u+1, j1-u:j1+u+1, :]
        m = testcube.reshape(w2, B)

        # NED
        center = m[(w2+1)//2, :]
        NED = np.sqrt(np.sum(((m / np.linalg.norm(m, axis=1, keepdims=True)) - (center / np.linalg.norm(center)))**2, axis=1))
        ind = np.argsort(NED)
        index = ind[:L]
        Id[:, k-1] = index
        Fea_cube[:, k-1, :] = m[index, :]

# T-SVD decomposition
rank = min(L, B)  # Ensure the rank is appropriate for the tensor dimensions
U, S, VT = tSVD(Fea_cube, rank)

# Ensure shapes are compatible for tProduct
print(f"Shapes before tProduct: U={U.shape}, S={S.shape}, VT={VT.shape}")
assert U.shape[1] == S.shape[0]
assert S.shape[1] == VT.shape[0]

C = tProduct(U, S, VT)
print(f"Shape after first tProduct (C): {C.shape}")

# Perform the final tProduct
Feacube_proc = C.reshape(L, W * H, B)
print(f"Shape after second tProduct (Feacube_proc): {Feacube_proc.shape}")

# SVM and RF based classification
Labels = img_gt.reshape(W*H)
Vectors = Feacube_proc.transpose(1, 0, 2).reshape(W*H, L*B)  # Use processed feature cube for SVM and RF

class_num = np.max(img_gt) - np.min(img_gt)
trainVectors, trainLabels, testVectors, testLabels = [], [], [], []

rng = np.random.default_rng()
Sam = 0.02  # Training sample ratio: 0.02 for IP, 0.01 for PU and MG

for k in range(1, class_num + 1):
    index = np.where(Labels == k)[0]
    perclass_num = len(index)
    Vectors_perclass = Vectors[index, :]
    c = rng.permutation(perclass_num)
    select_train = Vectors_perclass[c[:int(np.ceil(perclass_num * Sam))], :]
    train_index_k = index[c[:int(np.ceil(perclass_num * Sam))]]
    trainVectors.append(select_train)
    trainLabels.extend([k] * int(np.ceil(perclass_num * Sam)))

    select_test = Vectors_perclass[c[int(np.ceil(perclass_num * Sam)):], :]
    test_index_k = index[c[int(np.ceil(perclass_num * Sam)):]]
    testVectors.append(select_test)
    testLabels.extend([k] * (perclass_num - int(np.ceil(perclass_num * Sam))))

trainVectors = np.vstack(trainVectors)
testVectors = np.vstack(testVectors)

# Feature scaling
scaler = StandardScaler().fit(trainVectors)
trainVectors_scaled = scaler.transform(trainVectors)
testVectors_scaled = scaler.transform(testVectors)
Vectors_scaled = scaler.transform(Vectors)

# SVM classifier
svm_model = SVC(kernel='linear', C=1)
svm_model.fit(trainVectors_scaled, trainLabels)
svm_predictions = svm_model.predict(testVectors_scaled)

# Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(trainVectors_scaled, trainLabels)
rf_predictions = rf_model.predict(testVectors_scaled)

# Metrics
svm_accuracy = accuracy_score(testLabels, svm_predictions)
svm_kappa = cohen_kappa_score(testLabels, svm_predictions)
rf_accuracy = accuracy_score(testLabels, rf_predictions)
rf_kappa = cohen_kappa_score(testLabels, rf_predictions)

print(f"SVM Accuracy: {svm_accuracy}")
print(f"SVM Kappa: {svm_kappa}")
print(f"RF Accuracy: {rf_accuracy}")
print(f"RF Kappa: {rf_kappa}")

# Confusion Matrix
svm_cm = confusion_matrix(testLabels, svm_predictions)
rf_cm = confusion_matrix(testLabels, rf_predictions)

# Plotting the confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(svm_cm, interpolation='nearest', cmap=plt.cm.Blues)
axes[0].set_title('SVM Confusion Matrix')
axes[1].imshow(rf_cm, interpolation='nearest', cmap=plt.cm.Blues)
axes[1].set_title('RF Confusion Matrix')
plt.show()


Error loading the file: Unable to open file (truncated file: eof = 46974730, sblock->base_addr = 512, stored_eof = 473815497)
Using mock data for img since the file could not be loaded.


NameError: name 'W' is not defined

In [10]:
import h5py
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import matplotlib.pyplot as plt

def tSVD(tensor, rank):
    """ Tensor Singular Value Decomposition """
    unfold_tensor = tensor.reshape(tensor.shape[0], -1)
    U, S, VT = np.linalg.svd(unfold_tensor, full_matrices=False)
    U = U[:, :rank]
    S = np.diag(S)[:rank, :rank]
    VT = VT[:rank, :]
    return U, S, VT

def tProduct(U, S, VT):
    """ Tensor Product """
    result = np.tensordot(U, S, axes=(1, 0))
    result = np.tensordot(result, VT, axes=(1, 0))
    return result

# Load the provided dataset
file_path = "C:/Users/Mrlaptop/Downloads/Brain_Data_Mat_Format/Brain_Data_Mat_Format/008-01reflectance.mat"

data = {}
try:
    with h5py.File(file_path, 'r') as f:
        # Inspecting the keys to understand the structure of the data
        print(f.keys())
        # Assuming the dataset key is 'reflectance'
        # Update this with the actual key after inspecting the structure
        data = {key: np.array(f[key]) for key in f.keys()}
except Exception as e:
    print(f"Error loading the file: {e}")

# Ensure that the data was loaded
if 'reflectance' in data:
    img = data['reflectance']
else:
    print("Using mock data for img since the file could not be loaded.")
    img = np.random.rand(100, 100, 200)  # Mock data as an example
    img_gt = np.random.randint(1, 5, (100, 100))  # Mock ground truth

# If img_gt is not already defined (from the file), define a mock ground truth
if 'img_gt' not in locals():
    W, H, B = img.shape
    img_gt = np.random.randint(1, 5, (W, H))  # Assuming there are 4 classes

# Parameters (Can be changed as required)
u = 5
w = 2 * u + 1
w2 = w * w
L = 49

# TensorSSA
indian_pines = np.pad(img, ((u, u), (u, u), (0, 0)), mode='symmetric')
Id = np.zeros((L, W * H), dtype=int)
Fea_cube = np.zeros((L, W * H, B))

# Adaptive embedding
k = 0
for i in range(W):
    for j in range(H):
        i1 = i + u
        j1 = j + u
        k += 1
        testcube = indian_pines[i1-u:i1+u+1, j1-u:j1+u+1, :]
        m = testcube.reshape(w2, B)

        # NED
        center = m[(w2+1)//2, :]
        NED = np.sqrt(np.sum(((m / np.linalg.norm(m, axis=1, keepdims=True)) - (center / np.linalg.norm(center)))**2, axis=1))
        ind = np.argsort(NED)
        index = ind[:L]
        Id[:, k-1] = index
        Fea_cube[:, k-1, :] = m[index, :]

# T-SVD decomposition
rank = min(L, B)  # Ensure the rank is appropriate for the tensor dimensions
U, S, VT = tSVD(Fea_cube, rank)

# Ensure shapes are compatible for tProduct
print(f"Shapes before tProduct: U={U.shape}, S={S.shape}, VT={VT.shape}")
assert U.shape[1] == S.shape[0]
assert S.shape[1] == VT.shape[0]

C = tProduct(U, S, VT)
print(f"Shape after first tProduct (C): {C.shape}")

# Perform the final tProduct
Feacube_proc = C.reshape(L, W * H, B)
print(f"Shape after second tProduct (Feacube_proc): {Feacube_proc.shape}")

# SVM and RF based classification
Labels = img_gt.reshape(W*H)
Vectors = Feacube_proc.transpose(1, 0, 2).reshape(W*H, L*B)  # Use processed feature cube for SVM and RF

class_num = np.max(img_gt) - np.min(img_gt)
trainVectors, trainLabels, testVectors, testLabels = [], [], [], []

rng = np.random.default_rng()
Sam = 0.02  # Training sample ratio: 0.02 for IP, 0.01 for PU and MG

for k in range(1, class_num + 1):
    index = np.where(Labels == k)[0]
    perclass_num = len(index)
    Vectors_perclass = Vectors[index, :]
    c = rng.permutation(perclass_num)
    select_train = Vectors_perclass[c[:int(np.ceil(perclass_num * Sam))], :]
    train_index_k = index[c[:int(np.ceil(perclass_num * Sam))]]
    trainVectors.append(select_train)
    trainLabels.extend([k] * int(np.ceil(perclass_num * Sam)))

    select_test = Vectors_perclass[c[int(np.ceil(perclass_num * Sam)):], :]
    test_index_k = index[c[int(np.ceil(perclass_num * Sam)):]]
    testVectors.append(select_test)
    testLabels.extend([k] * (perclass_num - int(np.ceil(perclass_num * Sam))))

trainVectors = np.vstack(trainVectors)
testVectors = np.vstack(testVectors)

# Feature scaling
scaler = StandardScaler().fit(trainVectors)
trainVectors_scaled = scaler.transform(trainVectors)
testVectors_scaled = scaler.transform(testVectors)
Vectors_scaled = scaler.transform(Vectors)

# SVM classifier
svm_model = SVC(kernel='linear', C=1)
svm_model.fit(trainVectors_scaled, trainLabels)
svm_predictions = svm_model.predict(testVectors_scaled)

# Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(trainVectors_scaled, trainLabels)
rf_predictions = rf_model.predict(testVectors_scaled)

# Metrics
svm_accuracy = accuracy_score(testLabels, svm_predictions)
svm_kappa = cohen_kappa_score(testLabels, svm_predictions)
rf_accuracy = accuracy_score(testLabels, rf_predictions)
rf_kappa = cohen_kappa_score(testLabels, rf_predictions)

print(f"SVM Accuracy: {svm_accuracy}")
print(f"SVM Kappa: {svm_kappa}")
print(f"RF Accuracy: {rf_accuracy}")
print(f"RF Kappa: {rf_kappa}")

# Confusion Matrix
svm_cm = confusion_matrix(testLabels, svm_predictions)
rf_cm = confusion_matrix(testLabels, rf_predictions)

# Plotting the confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(svm_cm, interpolation='nearest', cmap=plt.cm.Blues)
axes[0].set_title('SVM Confusion Matrix')
axes[1].imshow(rf_cm, interpolation='nearest', cmap=plt.cm.Blues)
axes[1].set_title('RF Confusion Matrix')
plt.show()


Error loading the file: Unable to open file (truncated file: eof = 46974730, sblock->base_addr = 512, stored_eof = 473815497)
Using mock data for img since the file could not be loaded.


NameError: name 'W' is not defined

In [1]:
import h5py
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import matplotlib.pyplot as plt

def tSVD(tensor, rank):
    """ Tensor Singular Value Decomposition """
    unfold_tensor = tensor.reshape(tensor.shape[0], -1)
    U, S, VT = np.linalg.svd(unfold_tensor, full_matrices=False)
    U = U[:, :rank]
    S = np.diag(S)[:rank, :rank]
    VT = VT[:rank, :]
    return U, S, VT

def tProduct(U, S, VT):
    """ Tensor Product """
    result = np.tensordot(U, S, axes=(1, 0))
    result = np.tensordot(result, VT, axes=(1, 0))
    return result

# Load the provided dataset
file_path ="C:/Users/Mrlaptop/Downloads/Brain_Data_Mat_Format/Brain_Data_Mat_Format/004-02reflectance.mat"

data = {}
try:
    with h5py.File(file_path, 'r') as f:
        # Inspecting the keys to understand the structure of the data
        print(f.keys())
        # Assuming the dataset key is 'reflectance'
        data = {key: np.array(f[key]) for key in f.keys()}
except Exception as e:
    print(f"Error loading the file: {e}")

# Ensure that the data was loaded
if 'reflectance' in data:
    img = data['reflectance']
else:
    print("Unable to load the dataset.")
    exit()

# Mock ground truth
W, H, B = img.shape
img_gt = np.random.randint(1, 5, (W, H))  # Assuming there are 4 classes

# Parameters (Can be changed as required)
u = 5
w = 2 * u + 1
w2 = w * w
L = 49

# TensorSSA
indian_pines = np.pad(img, ((u, u), (u, u), (0, 0)), mode='symmetric')
Id = np.zeros((L, W * H), dtype=int)
Fea_cube = np.zeros((L, W * H, B))

# Adaptive embedding
k = 0
for i in range(W):
    for j in range(H):
        i1 = i + u
        j1 = j + u
        k += 1
        testcube = indian_pines[i1-u:i1+u+1, j1-u:j1+u+1, :]
        m = testcube.reshape(w2, B)

        # NED
        center = m[(w2+1)//2, :]
        NED = np.sqrt(np.sum(((m / np.linalg.norm(m, axis=1, keepdims=True)) - (center / np.linalg.norm(center)))**2, axis=1))
        ind = np.argsort(NED)
        index = ind[:L]
        Id[:, k-1] = index
        Fea_cube[:, k-1, :] = m[index, :]

# T-SVD decomposition
rank = min(L, B)  # Ensure the rank is appropriate for the tensor dimensions
U, S, VT = tSVD(Fea_cube, rank)

# Ensure shapes are compatible for tProduct
print(f"Shapes before tProduct: U={U.shape}, S={S.shape}, VT={VT.shape}")
assert U.shape[1] == S.shape[0]
assert S.shape[1] == VT.shape[0]

C = tProduct(U, S, VT)
print(f"Shape after first tProduct (C): {C.shape}")

# Perform the final tProduct
Feacube_proc = C.reshape(L, W * H, B)
print(f"Shape after second tProduct (Feacube_proc): {Feacube_proc.shape}")

# SVM and RF based classification
Labels = img_gt.reshape(W*H)
Vectors = Feacube_proc.transpose(1, 0, 2).reshape(W*H, L*B)  # Use processed feature cube for SVM and RF

class_num = np.max(img_gt) - np.min(img_gt)
trainVectors, trainLabels, testVectors, testLabels = [], [], [], []

rng = np.random.default_rng()
Sam = 0.02  # Training sample ratio: 0.02 for IP, 0.01 for PU and MG

for k in range(1, class_num + 1):
    index = np.where(Labels == k)[0]
    perclass_num = len(index)
    Vectors_perclass = Vectors[index, :]
    c = rng.permutation(perclass_num)
    select_train = Vectors_perclass[c[:int(np.ceil(perclass_num * Sam))], :]
    train_index_k = index[c[:int(np.ceil(perclass_num * Sam))]]
    trainVectors.append(select_train)
    trainLabels.extend([k] * int(np.ceil(perclass_num * Sam)))

    select_test = Vectors_perclass[c[int(np.ceil(perclass_num * Sam)):], :]
    test_index_k = index[c[int(np.ceil(perclass_num * Sam)):]]
    testVectors.append(select_test)
    testLabels.extend([k] * (perclass_num - int(np.ceil(perclass_num * Sam))))

trainVectors = np.vstack(trainVectors)
testVectors = np.vstack(testVectors)

# Feature scaling
scaler = StandardScaler().fit(trainVectors)
trainVectors_scaled = scaler.transform(trainVectors)
testVectors_scaled = scaler.transform(testVectors)
Vectors_scaled = scaler.transform(Vectors)

# SVM classifier
svm_model = SVC(kernel='linear', C=1)
svm_model.fit(trainVectors_scaled, trainLabels)
svm_predictions = svm_model.predict(testVectors_scaled)

# Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(trainVectors_scaled, trainLabels)
rf_predictions = rf_model.predict(testVectors_scaled)

# Metrics
svm_accuracy = accuracy_score(testLabels, svm_predictions)
svm_kappa = cohen_kappa_score(testLabels, svm_predictions)
rf_accuracy = accuracy_score(testLabels, rf_predictions)
rf_kappa = cohen_kappa_score(testLabels, rf_predictions)

print(f"SVM Accuracy: {svm_accuracy}")
print(f"SVM Kappa: {svm_kappa}")
print(f"RF Accuracy: {rf_accuracy}")
print(f"RF Kappa: {rf_kappa}")

# Confusion Matrix
svm_cm = confusion_matrix(testLabels, svm_predictions)
rf_cm = confusion_matrix(testLabels, rf_predictions)

# Plotting the confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(svm_cm, interpolation='nearest', cmap=plt.cm.Blues)
axes[0].set_title('SVM Confusion Matrix')
axes[1].imshow(rf_cm, interpolation='nearest', cmap=plt.cm.Blues)
axes[1].set_title('RF Confusion Matrix')
plt.show()


<KeysViewHDF5 ['gt', 'refl']>
Unable to load the dataset.


NameError: name 'img' is not defined